In [1]:
import pandas as pd
import numpy as np


from datetime import datetime, date, time, timedelta
import calendar as c         # c is the module
c.setfirstweekday(c.SUNDAY)  # first day Sunday America not Monday Euro
cc = c.Calendar()            # cc is the an instantiation of the class Calendar; class Calendar inside module c


from dateutil.relativedelta import *
from dateutil.rrule import *
from dateutil.parser import *


from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set(style='darkgrid')

from scipy import stats
from scipy.stats import mannwhitneyu

import copy


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_csv('../data/df_5.csv')
df_corpus_vile_non = copy.deepcopy(df)              # the backup in case of foobar

### The idea is that for 'overnight' for full moons effect hypothesis is that the official timestamp for a crime event must check for possible re-classification to a bucketed 'overnight' schema

#### Choose the at dinnertime 6 PM to start the overnight period;  cut the period at 3 AM so the full effect of Homicide 2:30 AM or so 'last call for homicide' can be included (It drops like a brick to the 7 AM rock bottom)

In [3]:
df['date_col'] = pd.to_datetime(df['date_col'])     # python various date-time stamps don't test for 'equal' well even
df['date'] = pd.to_datetime(df['date'])             #    they are 'equal', so the whole dance of converting and
df.index = df['date']                               #    comparing as strings seen in this notebook

In [4]:
df_moon_file = pd.read_csv('../data/moons_file.csv')

In [5]:
date_kernal = df['date_col']
date_kernal = list(set(date_kernal))     # get rid of some duplicates
dates_as_ymd_str_list = [str(each_date.to_period('D')) for each_date in date_kernal]
dates_as_ymd_str_list = sorted(list(set(dates_as_ymd_str_list)))  #  get rid of remaining duplicates and sort

In [6]:
# make a dictionary to hold each unique day as string form 'year-month-day', not as datetimes or any of their ilk

len(dates_as_ymd_str_list)

2559

In [7]:
values = [0] * 2559

overnight = dict(zip(dates_as_ymd_str_list, values)) 

In [8]:
# general idea: timestamp of today after 6 PM goes into the today bucket
#               timestamp of today before 3 AM goes into yesterday's bucket

def calc_6_to_3_overnight(row):                                 
    try:
        todays_date = row['date_col']
        todays_hour = row['hour_of_day']
        if todays_hour >= 18:
            overnight[str(todays_date.to_period('D'))] += 1
        if todays_hour <= 3:
            if (str(todays_date.to_period('D')) == '2012-11-15'):    # start date of the dataset does not have a
                pass                                                 #     yesterday; obvious only after it crashes
            else:                                                    #     on the edge case again
                yesterdays_date = todays_date - relativedelta(days=+1)
                overnight[str(yesterdays_date.to_period('D'))] += 1
    except Exception as e:
        print('Houston, we have a problem....')
        print(e)

In [9]:
df['overnight_6_to_3'] = df.apply(calc_6_to_3_overnight, axis= 1)

In [10]:
df_6_to_3 = pd.DataFrame(list(overnight.items()), columns= ['date_col', 'count'])
df_6_to_3.head(3)

,date_col,count
0,2012-11-15,374
1,2012-11-16,462
2,2012-11-17,445


In [11]:
df_moon_file.columns = ['date_col']
df_moon_file['flag_bit'] = 1
df_moon_file.head(3)

,date_col,flag_bit
0,2012-12-12,1
1,2013-01-21,1
2,2013-02-19,1


In [12]:
df_6_to_3 = pd.merge(df_6_to_3, df_moon_file, how= 'outer', on= 'date_col')

In [13]:
df_6_to_3['flag_bit'] = df_6_to_3['flag_bit'].fillna(0)

In [14]:
df_6_to_3['flag_bit'].sum()

82.0

In [15]:
df_6_to_3[df_6_to_3['flag_bit'] == 0].count()

date_col    2477
count       2477
flag_bit    2477
dtype: int64

In [16]:
df_6_to_3.shape

(2559, 3)

#####  The groups are unbalanced so cannot use t-test.   Use Mann Whitney test instead.

##### The scipy stats documentation states validity when n for both groups at least 20.    Requirements met.

###  Null Hypothesis

##### My Null Hypothesis is that there is no difference in the average number of crimes on the overnights when there is no full moon compared to the average number of crimes when there is a full moon.

In [17]:
#     H_0 :   mu_no_moon == mu_full_moon

In [18]:
#     My null hypothesis is that overnight crime samples are coming from a single population.
#     Run the collected re-bucketed data through stats models Mann Whitney

In [19]:
full_moon_here = list(df_6_to_3[df_6_to_3['flag_bit'] == 1]['count'])
full_moon_gone = list(df_6_to_3[df_6_to_3['flag_bit'] == 0]['count'])

In [20]:
stat, p = mannwhitneyu(full_moon_here, full_moon_gone)

In [21]:
p

0.2501860679603888

In [22]:
#   p > 0.05            NOT(Since p is low, H_0 gotta go)  ==>  cannot reject the Null Hypothesis

#### The null hypothesis is not rejected.  The purist would say the that's all you can say.

## I'll go out and say that the full moon does not have an effect on crime.